In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from data import classification_dataset, kfolds, train_test_split
from train import train_model, train_model_with_early_stopping
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
    SEED,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
def build_3d_cnn():
    input_small = keras.Input(SMALL_PATCH_SHAPE, name="input_small")
    x_small = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="small_selu_conv3d_1",
    )(input_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_1")(x_small)
    x_small = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="small_selu_conv3d_2",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_2")(x_small)
    x_small = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="small_selu_conv3d_3",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_3")(x_small)
    x_small = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="small_selu_conv3d_4",
    )(x_small)
    x_small = keras.layers.Flatten(name="flatten_small")(x_small)

    input_big = keras.Input(BIG_PATCH_SHAPE, name="input_big")
    x_big = keras.layers.MaxPooling3D((2, 2, 2), name="big_maxpool_0")(input_big)
    x_big = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="big_selu_conv3d_1",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_1")(x_big)
    x_big = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="big_selu_conv3d_2",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_2")(x_big)
    x_big = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="big_selu_conv3d_3",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_3")(x_big)
    x_big = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="big_selu_conv3d_4",
    )(x_big)
    x_big = keras.layers.Flatten(name="flatten_big")(x_big)

    x = keras.layers.concatenate([x_small, x_big], name="concatenate")
    x = keras.layers.Dense(1, activation="sigmoid", name="final_dense")(x)

    cnn_3d = keras.Model(inputs=[input_small, input_big], outputs=x, name="3dcnn")

    return cnn_3d

In [3]:
dataset, samples = classification_dataset(
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    return_size=True,
)
samples

754

In [4]:
val_perc = 0.1
batch_size = 16

In [5]:
train_dataset, val_dataset = train_test_split(
    dataset, test_perc=val_perc, cardinality=samples
)
val_dataset = val_dataset.batch(batch_size)
train_dataset = (
    train_dataset.cache()  # must be called before shuffle
    .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
    .batch(batch_size)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

<PrefetchDataset shapes: (((None, None, None, None, None), (None, None, None, None, None)), (None, 1)), types: ((tf.float32, tf.float32), tf.int8)>

In [6]:
learning_rate = 1e-5
patience = 15

In [7]:
cnn = build_3d_cnn()
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.MeanSquaredError(),
    metrics=[
        keras.metrics.AUC(name="auc"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
    ],
)

model_fname = f"models/lidc-3dcnn.h5"
log_dir = f"logs/lidc-3dcnn"
cnn = train_model_with_early_stopping(
    cnn,
    train_dataset,
    val_dataset,
    patience,
    "val_loss",
    model_fname,
    log_dir,
    verbose_checkpoint=1,
)


Epoch 00001: val_loss improved from inf to 0.18274, saving model to models/lidc-3dcnn.h5

Epoch 00002: val_loss improved from 0.18274 to 0.16303, saving model to models/lidc-3dcnn.h5

Epoch 00003: val_loss improved from 0.16303 to 0.15106, saving model to models/lidc-3dcnn.h5

Epoch 00004: val_loss improved from 0.15106 to 0.14526, saving model to models/lidc-3dcnn.h5

Epoch 00005: val_loss did not improve from 0.14526

Epoch 00006: val_loss improved from 0.14526 to 0.13689, saving model to models/lidc-3dcnn.h5

Epoch 00007: val_loss improved from 0.13689 to 0.13687, saving model to models/lidc-3dcnn.h5

Epoch 00008: val_loss improved from 0.13687 to 0.13308, saving model to models/lidc-3dcnn.h5

Epoch 00009: val_loss improved from 0.13308 to 0.12953, saving model to models/lidc-3dcnn.h5

Epoch 00010: val_loss improved from 0.12953 to 0.12910, saving model to models/lidc-3dcnn.h5

Epoch 00011: val_loss did not improve from 0.12910

Epoch 00012: val_loss improved from 0.12910 to 0.1249